# LangChain Tool Use Quickstart
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://python.langchain.com/v0.1/docs/use_cases/tool_use/quickstart/

LLM의 흥미로운 사용 사례 중 하나는 **API, 함수, 데이터베이스 등 다른 "도구"(tools)에 대한 자연어 인터페이스를 구축하는 것**입니다. LangChain은 이러한 인터페이스를 구축하는 데 매우 유용한데, 그 이유는 다음과 같습니다:








*   모델 출력 파싱이 우수하여 JSON, XML, OpenAI 함수 호출 등을 모델 출력에서 쉽게 추출할 수 있습니다.
*   다양한 내장 도구 모음 ( https://python.langchain.com/v0.1/docs/use_cases/tool_use/ )을 제공합니다.
*   이 도구들을 호출하는 방식에 있어 많은 유연성을 제공합니다.













도구를 사용하는 주요 방법에는 체인과 에이전트 두 가지가 있습니다.

체인은 도구 사용의 미리 정의된 순서를 만들 수 있게 합니다.

tool_1.svg

에이전트는 모델이 도구를 반복적으로 사용할 수 있게 하여, 도구를 몇 번 사용할지 결정할 수 있도록 합니다.

tool_2.svg

Quickstart 예제를 통해 두 가지 접근 방식을 살펴봅시다.

# Quickstart


이 가이드에서는 도구(Tools)를 호출하는 체인(Chains)과 에이전트(Agents)를 만드는 기본적인 방법을 다룹니다. 도구(Tools)는 API, 함수, 데이터베이스 등 거의 모든 것이 될 수 있습니다. 도구를 사용하면 모델의 기능을 단순히 텍스트나 메시지를 출력하는 것 이상의 것으로 확장할 수 있습니다. 도구를 사용하는 모델을 제대로 활용하려면 모델을 정확하게 프롬프트하고, 올바른 도구를 선택하여 적절한 입력을 제공하도록 모델의 응답을 분석하는 것이 중요합니다.








# 라이브러리 설치

In [ ]:
!pip install --upgrade --quiet langchain langchain-openai langchainhub

# LangSmith 설정 (Optional)

In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain Tool Use Quickstart - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LANGSMITH_API_KEY"

# Create a tool

먼저 호출할 도구(tool)를 만들어야 합니다. 이 예제에서는 함수로부터 사용자 정의 도구를 만들 것입니다. 사용자 정의 도구를 만드는 방법에 대한 자세한 내용은 이 가이드 ( https://python.langchain.com/v0.1/docs/modules/tools/ )를 참조하세요.








In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [ ]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

# Chains

도구를 정해진 횟수만큼만 사용해야 하는 경우, 이를 위해 체인을 만들 수 있습니다. 사용자 지정 숫자를 곱하기만 하는 간단한 체인을 만들어 보겠습니다.

# Tool/function calling

도구를 대형 언어 모델(LLM)과 함께 사용하는 가장 신뢰할 수 있는 방법 중 하나는 **도구 호출 API(때로는 함수 호출(function calling)이라고도 함)를 사용**하는 것입니다. 이는 **명시적으로 도구 호출을 지원하는 모델에서만 작동**합니다. 어떤 모델이 도구 호출을 지원하는지 여기( https://python.langchain.com/v0.1/docs/integrations/chat/ )에서 확인할 수 있으며, 도구 호출을 사용하는 방법에 대한 자세한 내용은 이 가이드( https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/ )에서 확인할 수 있습니다.

먼저 모델과 도구를 정의하겠습니다. 하나의 도구, 즉 multiply로 시작하겠습니다.

###  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_KEY)

모델에 대한 각 호출의 일부로 도구의 정의를 전달하기 위해 **bind_tools**를 사용하여, 모델이 적절할 때 도구를 호출할 수 있도록 하겠습니다:

In [ ]:
llm_with_tools = llm.bind_tools([multiply])

모델이 도구를 호출하면, 이는 출력의 AIMessage.tool_calls 속성에 나타납니다:

In [ ]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_8P2KOnQrQeYWzhylMSmsHfbR'}]

# Invoking the tool

좋습니다! 우리는 도구 호출을 생성할 수 있습니다. 하지만 실제로 도구를 호출하려면 어떻게 해야 할까요? 이를 위해 생성된 도구 인수를 도구에 전달해야 합니다. 간단한 예로, 첫 번째 도구 호출의 인수를 추출해보겠습니다:

In [ ]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

92

# Agents

체인은 특정 사용자 입력에 필요한 도구 사용 순서를 알고 있을 때 유용합니다. 하지만 일부 경우에는 도구를 몇 번 사용할지 입력에 따라 달라집니다. 이러한 경우에는 **도구를 몇 번 사용할지와 어떤 순서로 사용할지를 모델이 스스로 결정**하도록 하고 싶습니다. 에이전트를 사용하면 이를 구현할 수 있습니다.

LangChain에는 다양한 사용 사례에 최적화된 여러 내장 에이전트가 포함되어 있습니다. 모든 에이전트 유형에 대한 자세한 내용은 여기 ( https://python.langchain.com/v0.1/docs/modules/agents/agent_types/ )에서 확인하세요.

대부분의 사용 사례에서 가장 신뢰할 수 있고 권장되는 **도구 호출 에이전트(tool calling agent)**( https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/ )를 사용할 것입니다.










In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


에이전트는 여러 도구를 쉽게 사용할 수 있다는 점에서 매우 유용합니다. 여러 도구를 사용하는 체인을 만드는 방법에 대해 알아보려면 "여러 도구가 포함된 체인" 페이지 ( https://python.langchain.com/v0.1/docs/use_cases/tool_use/multiple_tools/ )를 확인하세요.

In [ ]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

에이전트를 사용하면 도구를 여러 번 사용할 수 있는 질문을 할 수 있습니다:

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking \(3\) to the fifth power and multiplying that by the sum of \(12\) and \(3\), then squaring the whole result is \(13,286,025\).

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking \\(3\\) to the fifth power and multiplying that by the sum of \\(12\\) and \\(3\\), then squaring the whole result is \\(13,286,025\\).'}